## Simulating noise on MPQP

Quantum computers hold immense potential, but a major hurdle is ``noise``. ``Noise`` refers to anything that disrupts a quantum computer's calculations.

In this Notebook, we introduce the concept of ``noise`` in quantum computing and how to run noisy simulations by defining noise models and adding then to a ``QCircuit``. We'll take a practical approach to describing how noise affects qubits in a computation and show how you can build `Depolarizing` noise channel. 

If you are not familiar with the basic concepts of noise for quantum systems, we recommand you this introduction `TODO`

### General imports


In [12]:
from mpqp.all import *
from mpqp.gates import *
from mpqp.noise import Depolarizing
from mpqp.measures import BasisMeasure
from mpqp.execution import *

## Types of Noise in Quantum Simulations

There are different noise types in quantum computing simulations but here we just focus on ``Depolarizing Noise``, which is a combination of bit-flip and phase-flip errors, causing the qubit to lose its quantum information.

### Depolarizing noise implemented in MPQP

##### Instantiate your noise model

In MPQP, the abstract class ``NoiseModel`` represents noisy channels acting on the qubits of the circuit, either after each gate application, or as an interaction with the environement (what is called idle noise). Each predefined noise model should extend this class, which has common attributes ``targets`` (indicating the indices of the qubits affected by this noise model) and the optional ``gates`` (indicating specific gates after which the noise will be applied)

For example, if one wants to apply a depolarizing noise on the circuit, he can use the class ``Depolarizing``. We must to specify as first argument the probability, or the error rate of the channel, the ``Depolarizing`` class can be applied to one or more qubits specified by the ``targets`` list. Additionally, the ``dimension`` parameter allows targeting specific gates within a quantum circuit. 

This flexibility allows researchers to simulate noise in various scenarios.

In [2]:
Depolarizing(0.5, [0, 1, 2]) # noise applied to qubit0, qubit1 and qubit2

Depolarizing(0.5, [0, 1, 2], 1)

``dimension`` is by default equal to 1.

In [3]:
Depolarizing(0.5, [0, 1, 2], dimension=2)
Depolarizing(0.5, [0, 1, 2], gates=[H, Rx, U])
Depolarizing(0.5, [0, 1, 2], dimension=2, gates=[CNOT, CZ]) # when the dimension is 2, the specified gates must be 2-qubit gates

Depolarizing(0.5, [0, 1, 2], 2, [CNOT, CZ])

### Adding noise to the circuit - several cases

In [2]:
# 1- using direct instantiation
circuit_1 = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100), Depolarizing(0.3, [0], gates=[H])])

In [3]:
circuit_1 # repr

QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure(0, 1, shots=100), Depolarizing(0.3, [0], 1, [H])], nb_qubits=2, nb_cbits=2, label="None")

In [4]:
print(circuit_1)

     ┌───┐          ┌─┐   
q_0: ┤ H ├──■───────┤M├───
     └───┘┌─┴─┐┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Y ├─╫─┤M├
          └───┘└───┘ ║ └╥┘
c: 2/════════════════╩══╩═
                     0  1 
NoiseModel: Depolarizing(0.3, [0], 1, [H])


In [5]:
circuit_1.pretty_print() # pretty print of the circuit

QCircuit : Size (Qubits,Cbits) = (2, 2), Nb instructions = 4
Depolarizing noise: probability 0.3 on qubits [0] for gates [H]
     ┌───┐          ┌─┐   
q_0: ┤ H ├──■───────┤M├───
     └───┘┌─┴─┐┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Y ├─╫─┤M├
          └───┘└───┘ ║ └╥┘
c: 2/════════════════╩══╩═
                     0  1 


In [7]:
# 2- using add method
circuit_2 = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100)]) # measurments must span the whole circuit
circuit_2.add([Depolarizing(0.3, [0]), Depolarizing(0.13, [1])])

In [8]:
print(circuit_1.noises) # check the applied noise types

[Depolarizing(0.3, [0], 1, [H])]


In [9]:
circuit_2.pretty_print()

QCircuit : Size (Qubits,Cbits) = (2, 2), Nb instructions = 4
Depolarizing noise: probability 0.3 on qubits [0]
Depolarizing noise: probability 0.13 on qubits [1]
     ┌───┐          ┌─┐   
q_0: ┤ H ├──■───────┤M├───
     └───┘┌─┴─┐┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Y ├─╫─┤M├
          └───┘└───┘ ║ └╥┘
c: 2/════════════════╩══╩═
                     0  1 


### without_noises
Remove the applied noise models from the circuit

In [10]:
print(circuit_1.without_noises())

     ┌───┐          ┌─┐   
q_0: ┤ H ├──■───────┤M├───
     └───┘┌─┴─┐┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Y ├─╫─┤M├
          └───┘└───┘ ║ └╥┘
c: 2/════════════════╩══╩═
                     0  1 


### to_other_language
Transforms the circuit into the corresponding circuit in the language specified.

Here, ``MPQP`` circuit into ``AWS-Braket`` circuit

In [13]:
noisy_braket_circuit = circuit_2.to_other_language(Language.BRAKET)
print(noisy_braket_circuit)

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.



T  : │         0         │         1          │         2          │
      ┌───┐ ┌───────────┐       ┌───────────┐                       
q0 : ─┤ H ├─┤ DEPO(0.3) ├───●───┤ DEPO(0.3) ├───────────────────────
      └───┘ └───────────┘   │   └───────────┘                       
                          ┌─┴─┐ ┌────────────┐ ┌───┐ ┌────────────┐ 
q1 : ─────────────────────┤ X ├─┤ DEPO(0.13) ├─┤ Y ├─┤ DEPO(0.13) ├─
                          └───┘ └────────────┘ └───┘ └────────────┘ 
T  : │         0         │         1          │         2          │


## Running on AWS devices

##### Get all supported devices 

In [23]:
for device in AWSDevice:
    print(device.name, "|", device.value)


BRAKET_LOCAL_SIMULATOR | LocalSimulator
BRAKET_SV1_SIMULATOR | quantum-simulator/amazon/sv1
BRAKET_DM1_SIMULATOR | quantum-simulator/amazon/dm1
BRAKET_TN1_SIMULATOR | quantum-simulator/amazon/tn1
BRAKET_IONQ_HARMONY | qpu/ionq/Harmony
BRAKET_IONQ_ARIA_1 | qpu/ionq/Aria-1
BRAKET_IONQ_ARIA_2 | qpu/ionq/Aria-2
BRAKET_IONQ_FORTE_1 | qpu/ionq/Forte-1
BRAKET_OQC_LUCY | qpu/oqc/Lucy
BRAKET_QUERA_AQUILA | qpu/quera/Aquila
BRAKET_RIGETTI_ASPEN_M_3 | qpu/rigetti/Aspen-M-3


##### Check only the noisy simulators

In [22]:
for device in AWSDevice:
    if device.is_noisy_simulator():
        print(device.name, "|", device.value)

BRAKET_LOCAL_SIMULATOR | LocalSimulator
BRAKET_SV1_SIMULATOR | quantum-simulator/amazon/sv1
BRAKET_DM1_SIMULATOR | quantum-simulator/amazon/dm1
BRAKET_TN1_SIMULATOR | quantum-simulator/amazon/tn1


### Run the circuit on a noisy simulator

In [16]:
# select the local noise simulator
device = AWSDevice.BRAKET_LOCAL_SIMULATOR

# run the circuit on the local simulator
result = run(circuit_2, device) # this line is valid for both noisy and non noisy cases

print(result)

Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [20, 34, 35, 11]
Probabilities: [0.2  0.34 0.35 0.11]
State: 00, Index: 0, Count: 20, Probability: 0.2
State: 01, Index: 1, Count: 34, Probability: 0.34
State: 10, Index: 2, Count: 35, Probability: 0.35
State: 11, Index: 3, Count: 11, Probability: 0.11
Error: None


